In [1]:
%load_ext autoreload
%autoreload 2
import sys
import torch
import pickle
import pandas as pd
from torch import nn
sys.path.append('..')
# sys.path.append('/system/user/beck/pwbeck/projects/regularization/ml_utilities')
from pathlib import Path
import matplotlib.pyplot as plt
from ml_utilities.torch_models.fc import FC
from erank.utils import load_model_from_epoch, load_directions_matrix_from_task_sweep, load_multiple_dir_matrices_from_sweep
from erank.plot import plot_models_erank
from erank.analyses import create_model_erank_df

gpu_id = 0

## Erank plots

### Create model dict

In [2]:
sweep_dir = '/system/user/beck/pwbeck/projects/regularization/erank/outputs/sl-sine-10.0.1-sl-randinit-ampl-XXX-ph-XXX-taskidx-XXX_220912_234805'
sweep_output_path = Path(sweep_dir) / 'outputs'
num_vectors=1900

In [3]:
# check for missing best_epoch_files
# for p in sweep_output_path.iterdir():
#     bepf = p / 'best_epoch.txt'
#     if not bepf.exists():
#         print(str(p))

In [4]:
sine_task_distr = {'normal': '*ampl-0.1_5.0-ph-0_3.141592654*', 
                    'wide': '*ampl-0.1_10.0-ph-0_3.141592654*',
                    'narrow': '*ampl-0.1_2.5-ph-0_1.5*',
                    'almost_point': '*ampl-1.0_1.1-ph-0_0.1*'}

In [5]:
len(sine_task_distr)

4

In [6]:
# load a model
# run = next(iter(sweep_output_path.iterdir()))
# init_model = load_model_from_epoch(run, epoch=0, device='cpu')

In [7]:
# load model matrices
# model_dict = load_multiple_dir_matrices_from_sweep(path_to_runs=sweep_output_path,
#                                                    name_run_glob_pattern_dict=sine_task_distr,
#                                                    num_runs=num_vectors,
#                                                    device='cpu')
# with open(f'model_dict_num_vecs{num_vectors}.p', 'wb') as savefile:
#     pickle.dump(model_dict, savefile, protocol=pickle.HIGHEST_PROTOCOL)

### Load model dict

In [8]:
model_dict_path = './../tmp/erank_plots/model_dict_num_vecs1900.p'
model_dict_path = Path(model_dict_path)
init_model_path = './../tmp/erank_plots/init_model.p'
init_model_path = Path(init_model_path)

In [9]:
# load init model
init_model = FC.load(init_model_path, device=gpu_id)
init_model, init_model.device

(FC(
   (fc): Sequential(
     (0): Flatten(start_dim=1, end_dim=-1)
     (1): Linear(in_features=1, out_features=40, bias=True)
     (2): ReLU(inplace=True)
     (3): Linear(in_features=40, out_features=40, bias=True)
     (4): ReLU(inplace=True)
     (5): Linear(in_features=40, out_features=1, bias=True)
   )
 ),
 device(type='cuda', index=0))

In [10]:
# unpickle model dict
with open(model_dict_path, 'rb') as loadfile:
    model_dict = pickle.load(loadfile)

In [11]:
list(model_dict.keys()), type(model_dict)

(['normal#*ampl-0.1_5.0-ph-0_3.141592654*',
  'wide#*ampl-0.1_10.0-ph-0_3.141592654*',
  'narrow#*ampl-0.1_2.5-ph-0_1.5*',
  'almost_point#*ampl-1.0_1.1-ph-0_0.1*'],
 dict)

### Generate erank plot

In [12]:
erank_df = create_model_erank_df(models=model_dict, random_init_model=init_model, device=gpu_id)

Calculate eranks: 100%|██████████| 191/191 [02:06<00:00,  1.51it/s]


In [34]:
for c in erank_df:
    print(c)

normal#*ampl-0.1_5.0-ph-0_3.141592654*
wide#*ampl-0.1_10.0-ph-0_3.141592654*
narrow#*ampl-0.1_2.5-ph-0_1.5*
almost_point#*ampl-1.0_1.1-ph-0_0.1*
Gaussian random
Random model initializations


In [44]:
# erank_df.to_pickle('./erank_df.p')

In [45]:
# pd.read_pickle('./erank_df.p')

,normal#*ampl-0.1_5.0-ph-0_3.141592654*,wide#*ampl-0.1_10.0-ph-0_3.141592654*,narrow#*ampl-0.1_2.5-ph-0_1.5*,almost_point#*ampl-1.0_1.1-ph-0_0.1*,Gaussian random,Random model initializations
Number of Vectors,,,,,,
1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
11,8.514350,8.536983,7.899185,6.776556,10.991661,10.845201
21,15.354296,15.209914,14.060792,11.512559,20.969368,20.581127
31,21.480988,21.461157,19.562881,15.642528,30.933449,30.276787
41,27.208231,27.214802,24.733379,19.151588,40.873451,39.941334
...,...,...,...,...,...,...
1861,262.816956,279.277802,177.222214,105.386322,1450.250244,1374.447632
1871,263.001129,279.629730,177.338623,105.438393,1453.153442,1376.729614
1881,263.307922,279.700653,177.480469,105.575989,1455.965942,1378.998291


In [39]:
import os
from matplotlib import rc
os.environ["PATH"] = "/system/user/beck/.texlive2018/bin/x86_64-linux" + os.pathsep + os.environ["PATH"] 
rc('text', usetex=True)
fig, ax = plot_models_erank(erank_df=erank_df)

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f0795cfb880> (for post_execute):


RuntimeError: latex was not able to process the following string:
b'$\\\\mathdefault{750}$'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.19 (TeX Live 2018) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(../f752d3969d363fa1bd6dc6215508ccf7.tex
LaTeX2e <2017-04-15>
Babel <3.17> and hyphenation patterns for 3 language(s) loaded.
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2014/09/29 v1.4h Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))

! LaTeX Error: File `type1cm.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.5 \usepackage
               {type1ec}^^M
No pages of output.
Transcript written on f752d3969d363fa1bd6dc6215508ccf7.log.




RuntimeError: latex was not able to process the following string:
b'$\\\\mathdefault{750}$'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.19 (TeX Live 2018) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(../f752d3969d363fa1bd6dc6215508ccf7.tex
LaTeX2e <2017-04-15>
Babel <3.17> and hyphenation patterns for 3 language(s) loaded.
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2014/09/29 v1.4h Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))

! LaTeX Error: File `type1cm.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.5 \usepackage
               {type1ec}^^M
No pages of output.
Transcript written on f752d3969d363fa1bd6dc6215508ccf7.log.




<Figure size 944.882x393.701 with 1 Axes>

In [ ]:
fig.savefig(f'sinus_erank_varying_task_distribution_num_vecs{num_vectors}.pdf', bbox_inches='tight')

In [ ]:
fig.savefig(f'sinus_erank_varying_task_distribution_num_vecs{num_vectors}.png', dpi=300, bbox_inches='tight')